# Projeto de Precificação de Imóveis de São Paulo

## Etapa 1: Extração e Tratamento dos dados

Na primeira etapa do projeto, será realizado a extração e tratamento das bases de dados dos imóveis do município de São Paulo de modo a criar uma base de dados consistente para ser utilizada na etapa de análise exploratória, são eles:

* Dados do valor dos imovéis (Fonte: Kaggle)
* Dados do censo imobiliário (Fonte: IBGE)
* Dados dos endereços 
* Dados geográficos (Fonte: IBGE)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
from shapely.geometry import Point, Polygon

## Tratamento

### Dados do valor dos imóveis

In [3]:
path_aula = 'https://gist.githubusercontent.com/tgcsantos/3bdb29eba6ce391e90df2b72205ba891/raw/22fa920e80c9fa209a9fccc8b52d74cc95d1599b/dados_imoveis.csv'

In [4]:
path_imovel = "https://raw.githubusercontent.com/rafaelladuarte/precificacao_imoveis/main/data_properties_SP.csv"
df_imovel = pd.read_csv(path_imovel)
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price
1115,Rua Correia de Almeida,"Jabaquara, São Paulo",240,3,3,3,R$ 990.000
8852,"Bosque da Saúde, São Paulo",SP,135,2,1,1,R$ 669.000
6769,"Jardim Europa, São Paulo",SP,1317,4,5,7,R$ 25.000.000
9324,Rua Jaguanambi,"Cidade Jardim, São Paulo",589,4,6,6,R$ 4.000.000
6514,Rua Jorge Tibiriçá,"Vila Mariana, São Paulo",300,6,4,2,R$ 1.600.000


Limpando os espaços vazios extras nos valores dos campos

In [5]:
df_imovel = df_imovel.applymap(
    lambda x: x.strip() if isinstance(x, str) else x
)

Nota-se que o dataframe não possui a coluna que nos diz a respeito em qual bairro da cidade de São Paulo o imóvel localiza. No entanto, é possivel observar que essa informação pode ser encontrada na coluna Street ou City.

Ao separar os valores do campo Street por virgula, nota-se que pode ocorrer 3 partições

In [6]:
df_imovel["Street"].str.split(pat=",",expand = True)

,0,1,2
0,Avenida Itacira,255,None
1,Rua Aurelia Perez Alvarez,42,None
2,Rua Alba Valdez,None,None
3,Jardim Morumbi,São Paulo,None
4,Rua Tobias Barreto,195,None
...,...,...,...
10028,Rua Tapuçu,None,None
10029,Rua Guararema,None,None
10030,Rua Estero Belaco,None,None
10031,Rua Manuel Onha,,514


Inserindo as partições da separação no dataframe

In [7]:
df_imovel[
    [
        "Street_1",
        "Street_2", 
        "Street_3"
    ]
]  = df_imovel["Street"].str.split(pat=",",expand = True)

Visualizando as partições no dataframe

In [8]:
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3
4810,Rua das Jabuticabeiras,"Cidade Jardim, São Paulo",776,4,4,4,R$ 17.000\n /Mês,Rua das Jabuticabeiras,None,None
1183,Rua Alfredo Piragibe,"Vila Madalena, São Paulo",464,5,7,3,R$ 2.800.000,Rua Alfredo Piragibe,None,None
371,Rua Pedro Madureira,"Jardim São Paulo(Zona Norte), São Paulo",270,3,3,3,R$ 1.300.000,Rua Pedro Madureira,None,None
9136,"Rua Baronesa de Bela Vista, 138","Vila Congonhas, São Paulo",290,4,4,4,R$ 2.000.000,Rua Baronesa de Bela Vista,138,None
9873,Rua Quirino Pucca,"Vila Romana, São Paulo",80,1,1,2,R$ 480.000,Rua Quirino Pucca,None,None


Já ao separar os valores do campo City por virgula, ocorrer apenas 2 partições

In [9]:
df_imovel["City"].str.split(pat=",",expand = True)

,0,1
0,Planalto Paulista,São Paulo
1,Jardim dos Estados,São Paulo
2,Jardim Reimberg,São Paulo
3,SP,None
4,Mooca,São Paulo
...,...,...
10028,Vila Sofia,São Paulo
10029,Bosque da Saúde,São Paulo
10030,Vila da Saúde,São Paulo
10031,Vila Oratório,São Paulo


Inserindo as partições da separação no dataframe

In [10]:
df_imovel[["City_1", "City_2"]]  = df_imovel["City"].str.split(pat=",",expand = True)

Visualizando as partições no dataframe

In [11]:
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2
7251,"Pacaembu, São Paulo",SP,450,4,3,5,R$ 12.000\n /Mês,Pacaembu,São Paulo,None,SP,None
2399,"Jardim Paulista, São Paulo",SP,935,5,6,4,R$ 12.000.000,Jardim Paulista,São Paulo,None,SP,None
65,"Avenida João XXIII, 925","Vila Formosa, São Paulo",225,2,2,2,R$ 780.000,Avenida João XXIII,925,None,Vila Formosa,São Paulo
5894,"Jardim Bélgica, São Paulo",SP,220,3,5,4,R$ 1.590.000,Jardim Bélgica,São Paulo,None,SP,None
5957,Rua Ucaiari,"Alto da Lapa, São Paulo",143,3,5,2,R$ 1.200.000,Rua Ucaiari,None,None,Alto da Lapa,São Paulo


Atualizando o campo City para São Paulo, cidade dos dados que está sendo analisado

In [12]:
df_imovel["City"] = "São Paulo"

In [13]:
df_imovel.sample(2)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2
6139,Rua Polônia,São Paulo,486,4,4,3,R$ 3.190.000,Rua Polônia,None,None,Jardim Europa,São Paulo
5540,Rua Luís Góis,São Paulo,100,4,2,3,R$ 650.000,Rua Luís Góis,None,None,Mirandópolis,São Paulo


Criando a coluna Bairro

In [14]:
df_imovel["Bairro"] = df_imovel["City_1"]

Removendo valores do campo Street, quando a informação representa um bairro

In [15]:
df_imovel.loc[
    df_imovel["Bairro"] == "SP", "Street"
] = np.nan

Inserindo o valor do Bairro na sua respectiva coluna

In [16]:
df_imovel.loc[
    df_imovel["Bairro"] == "SP", "Bairro"
] = df_imovel['Street_1']

In [17]:
df_imovel.head()

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2,Bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Avenida Itacira,255,None,Planalto Paulista,São Paulo,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Rua Aurelia Perez Alvarez,42,None,Jardim dos Estados,São Paulo,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Rua Alba Valdez,None,None,Jardim Reimberg,São Paulo,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi,São Paulo,None,SP,None,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Rua Tobias Barreto,195,None,Mooca,São Paulo,Mooca


Removendo as colunas desnecessarias

In [18]:
df_imovel = df_imovel.drop(
    columns = [
        'Street_1',
        'Street_2',	
        'Street_3',
        'City_1',
        'City_2',
    ]
)
df_imovel.head()

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca


In [19]:
df_imovel.columns = ['Rua', 'cidade','metragem','quartos','banheiros','vagas','preco','bairro']

In [20]:
df_imovel.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca


Tratando os valores da coluna preço

Nota-se que elas estão no tipo string

In [21]:
type(df_imovel["preco"][0])

str

Separando a string por espaço

In [22]:
df_imovel["preco"].str.split(expand = True)

,0,1,2
0,R$,7.000.000,None
1,R$,3.700.000,None
2,R$,380.000,None
3,R$,685.000,None
4,R$,540.000,None
...,...,...,...
10028,R$,665.000,None
10029,R$,2.300.000,None
10030,R$,1.050.000,None
10031,R$,2.200,/Mês


In [23]:
df_imovel["preco"].str.split(expand = True)[2].unique()

array([None, '/Mês', '/Ano', '/Dia'], dtype=object)

In [24]:
df_imovel[["moeda", "preco_anuncio", "tipo_anuncio"]] = df_imovel["preco"].str.split(expand = True)

In [25]:
df_imovel.sample(3)

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio
9710,Rua Gregório Lopes,São Paulo,280,3,5,8,R$ 1.250.000,Parque Mandaqui,R$,1.250.000,None
7543,"Rua Milton Soares, 91",São Paulo,100,2,2,2,R$ 350.000,Jardim Sarah,R$,350.000,None
1400,Rua Ibiapinópolis,São Paulo,186,3,2,2,R$ 3.280.000,Jardim Paulistano,R$,3.280.000,None


In [26]:
df_imovel[df_imovel["tipo_anuncio"].isnull()]["tipo_anuncio"].unique()

array([None], dtype=object)

In [27]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10033 entries, 0 to 10032
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rua            6596 non-null   object
 1   cidade         10033 non-null  object
 2   metragem       10033 non-null  object
 3   quartos        10033 non-null  object
 4   banheiros      10033 non-null  object
 5   vagas          10033 non-null  object
 6   preco          10033 non-null  object
 7   bairro         10033 non-null  object
 8   moeda          10033 non-null  object
 9   preco_anuncio  10033 non-null  object
 10  tipo_anuncio   852 non-null    object
dtypes: object(11)
memory usage: 862.3+ KB


In [28]:
df_imovel = df_imovel[df_imovel["tipo_anuncio"].isnull()]
df_imovel

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista,R$,7.000.000,None
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados,R$,3.700.000,None
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg,R$,380.000,None
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi,R$,685.000,None
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca,R$,540.000,None
...,...,...,...,...,...,...,...,...,...,...,...
10027,Avenida Odila,São Paulo,262,3,6,4,R$ 1.500.000,Planalto Paulista,R$,1.500.000,None
10028,Rua Tapuçu,São Paulo,90,2,2,2,R$ 665.000,Vila Sofia,R$,665.000,None
10029,Rua Guararema,São Paulo,600,8,7,5,R$ 2.300.000,Bosque da Saúde,R$,2.300.000,None
10030,Rua Estero Belaco,São Paulo,200,3,3,6,R$ 1.050.000,Vila da Saúde,R$,1.050.000,None


In [29]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rua            6129 non-null   object
 1   cidade         9181 non-null   object
 2   metragem       9181 non-null   object
 3   quartos        9181 non-null   object
 4   banheiros      9181 non-null   object
 5   vagas          9181 non-null   object
 6   preco          9181 non-null   object
 7   bairro         9181 non-null   object
 8   moeda          9181 non-null   object
 9   preco_anuncio  9181 non-null   object
 10  tipo_anuncio   0 non-null      object
dtypes: object(11)
memory usage: 860.7+ KB


Substituindo o ponto dos milhares para vazio, pra conseguir converter os valores para float

In [30]:
df_imovel["preco_anuncio"].str.replace(".","",regex=False).astype(float)

0        7000000.0
1        3700000.0
2         380000.0
3         685000.0
4         540000.0
           ...    
10027    1500000.0
10028     665000.0
10029    2300000.0
10030    1050000.0
10032     270000.0
Name: preco_anuncio, Length: 9181, dtype: float64

Adicionando uma coluna no dataframe para o preço do anuncio em formato float

In [31]:
df_imovel["preco_anuncio_float"] = df_imovel["preco_anuncio"].str.replace(".","",regex=True).astype(float)

In [32]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rua                  6129 non-null   object 
 1   cidade               9181 non-null   object 
 2   metragem             9181 non-null   object 
 3   quartos              9181 non-null   object 
 4   banheiros            9181 non-null   object 
 5   vagas                9181 non-null   object 
 6   preco                9181 non-null   object 
 7   bairro               9181 non-null   object 
 8   moeda                9181 non-null   object 
 9   preco_anuncio        9181 non-null   object 
 10  tipo_anuncio         0 non-null      object 
 11  preco_anuncio_float  9181 non-null   float64
dtypes: float64(1), object(11)
memory usage: 932.4+ KB


In [33]:
df_imovel.sample(5)

,Rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio,preco_anuncio_float
3523,Avenida Doutor Arnaldo,São Paulo,270,3,3,4,R$ 1.950.000,Sumaré,R$,1.950.000,None,1950000.0
4853,Rua Santa Bibiana,São Paulo,250,3,4,6,R$ 1.100.000,Vila Sônia,R$,1.100.000,None,1100000.0
2756,NaN,São Paulo,100,3,2,3,R$ 480.000,Parque Residencial Oratorio,R$,480.000,None,480000.0
4677,Rua Juvenal Galeno,São Paulo,220,3,3,3,R$ 1.068.000,Jardim da Saúde,R$,1.068.000,None,1068000.0
5278,Rua Maracaí,São Paulo,400,4,2,4,R$ 2.500.000,Aclimação,R$,2.500.000,None,2500000.0


Removendo as colunas desnecessarias

In [34]:
df_imovel = df_imovel.drop(
    columns = [
        'tipo_anuncio',
        'moeda',
        'preco_anuncio',
        'preco'
    ]
)

In [35]:
df_imovel['metragem']

0           1000
1        469-524
2            125
3            310
4            100
          ...   
10027        262
10028         90
10029        600
10030        200
10032        117
Name: metragem, Length: 9181, dtype: object

Substituindo os valores que não são numeros nas colunas de metragem e vagas para NaN que significa Not a Number

In [36]:
df_imovel['metragem'] = df_imovel['metragem'].apply(
    lambda x: np.nan if '-' in x else x
)
df_imovel['metragem']

0        1000
1         NaN
2         125
3         310
4         100
         ... 
10027     262
10028      90
10029     600
10030     200
10032     117
Name: metragem, Length: 9181, dtype: object

In [37]:
df_imovel['vagas'] = df_imovel['vagas'].apply(
    lambda x: np.nan if '-' in x else x
)
df_imovel['vagas']

0        6
1        4
2        2
3        4
4        2
        ..
10027    4
10028    2
10029    5
10030    6
10032    2
Name: vagas, Length: 9181, dtype: object

Removendo as linhas que possuem valores nulos nas colunas de metragem e vagas

In [38]:
df_imovel = df_imovel.dropna(subset=["metragem","vagas"])
df_imovel

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0
2,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0
3,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0
5,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0
...,...,...,...,...,...,...,...,...
10027,Avenida Odila,São Paulo,262,3,6,4,Planalto Paulista,1500000.0
10028,Rua Tapuçu,São Paulo,90,2,2,2,Vila Sofia,665000.0
10029,Rua Guararema,São Paulo,600,8,7,5,Bosque da Saúde,2300000.0
10030,Rua Estero Belaco,São Paulo,200,3,3,6,Vila da Saúde,1050000.0


In [39]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9177 entries, 0 to 10032
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rua                  6125 non-null   object 
 1   cidade               9177 non-null   object 
 2   metragem             9177 non-null   object 
 3   quartos              9177 non-null   object 
 4   banheiros            9177 non-null   object 
 5   vagas                9177 non-null   object 
 6   bairro               9177 non-null   object 
 7   preco_anuncio_float  9177 non-null   float64
dtypes: float64(1), object(7)
memory usage: 645.3+ KB


Mudando o tipo dos valores das colunas metragem, quartos, banheiros e vagas para inteiro

In [40]:
df_imovel["metragem"] = df_imovel["metragem"].astype(int)
df_imovel["quartos"] = df_imovel["quartos"].astype(int)
df_imovel["banheiros"] = df_imovel["banheiros"].astype(int)
df_imovel["vagas"] = df_imovel["vagas"].astype(int)

In [41]:
df_imovel.sample(5)

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float
596,"Rua Realengo, 185",São Paulo,127,3,2,2,Vila Madalena,960000.0
1155,Rua Murtinho Nobre,São Paulo,1181,8,10,20,Butantã,4300000.0
3609,NaN,São Paulo,100,2,2,2,Vila Mafra,330000.0
4584,NaN,São Paulo,900,4,8,6,Jardim América,14000000.0
4911,Rua Choneco,São Paulo,210,3,1,4,Vila Ré,840000.0


Removendo caracter especial do campo Rua

In [42]:
df_imovel["apenas_rua"] = df_imovel["Rua"].str.extract(r'(^[\w ]+)')
df_imovel["apenas_rua"] = df_imovel["apenas_rua"].str.lower().str.strip()
df_imovel.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira
2,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez
3,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto
5,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell


Exportando o dataframe dos valores dos imoveis para um arquivo csv

In [43]:
df_imovel.to_csv('dados_imoveis_tratados.csv')

### Dados do setor imobiliário

Nessa base de dados encontramos os dados referentes ao censo do setor imobiliário, com o codigo de cada setor regional

In [44]:
path_setor = 'https://gist.githubusercontent.com/tgcsantos/85f8c7b0a2edbc3e27fcad619b37d886/raw/a4954781e6bca9cb804062a3eea0b3b84679daf4/Basico_SP1.csv'
df_setor = pd.read_csv(path_setor)
df_setor.sample(2)

,,,,,,,,,,Cod_setor;Cod_Grandes Regi�es;Nome_Grande_Regiao;Cod_UF;Nome_da_UF ;Cod_meso;Nome_da_meso;Cod_micro;Nome_da_micro;Cod_RM;Nome_da_RM;Cod_municipio;Nome_do_municipio;Cod_distrito;Nome_do_distrito;Cod_subdistrito;Nome_do_subdistrito;Cod_bairro;Nome_do_bairro;Situacao_setor;Tipo_setor;V001;V002;V003;V004;V005;V006;V007;V008;V009;V010;V011;V012;
"355030829000123;3;""Regi�o Sudeste"";35;""S�o Paulo"";3515;""Metropolitana de S�o Paulo"";35061;""S�o Paulo"";20;""RM S�o Paulo"";3550308;""S�O PAULO"";355030829;""FREGUESIA DO �"";35503082900;""FREGUESIA DO �"";3550308000;""S�O PAULO (todos os setores)"";1;0;153;509;3",33;3,38;1464,56;5580757,14;1915,19;6442230,27;1066,25;3291983,39;1681,4;4159929,71.0
"355030807000043;3;""Regi�o Sudeste"";35;""S�o Paulo"";3515;""Metropolitana de S�o Paulo"";35061;""S�o Paulo"";20;""RM S�o Paulo"";3550308;""S�O PAULO"";355030807;""BELA VISTA"";35503080700;""BELA VISTA"";3550308000;""S�O PAULO (todos os setores)"";1;0;300;561;1",87;1,04;2372,19;5322518,05;2550,74;5267501,87;1885,18;4430322,02;2298,37;4451732,97.0


Nota-se que o tivemos problemas ao ler o arquivo, para resolver isso, primeiro determinamos o encoding para tratar os caracteres especiais, em seguida, determinamos o separador de colunas e delimitador de texto utilizado no arquivo.

In [45]:
df_setor = pd.read_csv(
    path_setor,
    encoding='ISO-8859-1',
    sep=';', thousands='.', decimal=','
)
df_setor.sample(5)

,Cod_setor,Cod_Grandes Regiï¿½es,Nome_Grande_Regiao,Cod_UF,Nome_da_UF,Cod_meso,Nome_da_meso,Cod_micro,Nome_da_micro,Cod_RM,...,V004,V005,V006,V007,V008,V009,V010,V011,V012,Unnamed: 33
2704,355030819000118,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,1.85,1134.17,5440788.36,1344.80,6171440.77,704.29,2302330.42,1137.69,3228160.43,NaN
4362,355030825000332,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.42,382.50,212318.02,645.92,188158.55,313.99,217802.00,715.67,208920.02,NaN
2149,355030817000098,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,2.74,886.15,847225.80,977.48,845264.57,543.78,574608.77,847.65,638218.82,NaN
12201,355030864000093,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,2.45,1299.49,2365473.05,1665.71,2422610.26,1011.61,1769421.97,1448.20,1901257.55,NaN
14140,355030873000213,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,1.64,1537.29,1297232.23,1622.10,1230765.16,854.61,1008133.18,1309.00,949148.61,NaN


In [46]:
df_setor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18363 entries, 0 to 18362
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Cod_setor              18363 non-null  int64  
 1   Cod_Grandes Regiï¿½es  18363 non-null  int64  
 2   Nome_Grande_Regiao     18363 non-null  object 
 3   Cod_UF                 18363 non-null  int64  
 4   Nome_da_UF             18363 non-null  object 
 5   Cod_meso               18363 non-null  int64  
 6   Nome_da_meso           18363 non-null  object 
 7   Cod_micro              18363 non-null  int64  
 8   Nome_da_micro          18363 non-null  object 
 9   Cod_RM                 18363 non-null  int64  
 10  Nome_da_RM             18363 non-null  object 
 11  Cod_municipio          18363 non-null  int64  
 12  Nome_do_municipio      18363 non-null  object 
 13  Cod_distrito           18363 non-null  int64  
 14  Nome_do_distrito       18363 non-null  object 
 15  Co

Quando vemos as informações gerais do dataframe gerado, vemos que ele possue muitos valores nulos. Desta forma, vamos trata-los removendo.

In [47]:
df_setor.dropna(how='all', axis=1, inplace=True)
df_setor.head()

,Cod_setor,Cod_Grandes Regiï¿½es,Nome_Grande_Regiao,Cod_UF,Nome_da_UF,Cod_meso,Nome_da_meso,Cod_micro,Nome_da_micro,Cod_RM,...,V003,V004,V005,V006,V007,V008,V009,V010,V011,V012
0,355030801000001,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.01,2.32,2091.32,8532451.78,2164.55,8673276.78,1227.41,4285771.99,1713.75,5152087.86
1,355030801000002,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,2.98,2.09,1650.09,3983441.31,1717.44,4030519.99,1045.78,2572133.32,1468.08,2991546.94
2,355030801000003,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.31,3.08,1280.43,5474593.06,2349.53,7553913.02,902.97,2735492.52,1786.79,3837674.74
3,355030801000004,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.16,2.26,1119.59,3572955.10,1859.13,4567289.50,869.09,2122409.02,1567.41,2735383.24
4,355030801000005,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.14,2.35,1416.90,2240207.58,1581.66,2240109.39,917.71,1570047.75,1356.35,1725877.85


### Dados dos endereços 

Nessa base encontramos os principais endereços existentes nos diversos estados do Brasil, cada um com sua geolocalização (latitude, longitude)

In [48]:
path_endereco = "/home/Rafaella.Duarte/Documentos/pasta_pessoal/precificacao_imoveis/Imersao_Dados/Large_Files/enderecos.csv"
df_endereco = pd.read_csv(path_endereco,low_memory=False)
df_endereco.sample(2)

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude
175007,Avenida,Beira-Rio,Avenida Beira-Rio,NaN,Distrito Industrial Simão da Cunha,Santa Luzia,MG,33040260.0,-19.783092,-43.857937
172598,Rua,Dinamarca,Rua Dinamarca,NaN,Conjunto Henrique Sapori,Ribeirão das Neves,MG,33823060.0,-19.764920,-44.140778


Como estamos analisando apenas os imovéis do municipio de São Paulo, em seguida iremos filtrar e seleciona apenas os dados desejados.

In [49]:
df_endereco = df_endereco.query("sigla_uf == 'SP'")
df_endereco.head()

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude
1,Rua,João Antônio Vasconcelos,Rua João Antônio Vasconcelos - até 627/628,627,Amizade,Araçatuba,SP,16074320.0,-21.190276,-50.448686
413269,Rua,A,Rua A,NaN,Jardim Glória,Americana,SP,13468220.0,-22.756531,-47.343738
413270,Rua,A,Rua A,NaN,Jardim São Roque,Americana,SP,13469414.0,-22.743734,-47.348631
413271,Rua,A,Rua A,NaN,Iate Clube de Campinas,Americana,SP,13475620.0,-22.737846,-47.333569
413272,Rua,João Bandini,Rua João Bandini,NaN,Jardim Miriam,Americana,SP,13469072.0,-22.744283,-47.351588


Adicionando um coluna com o a rua dos endereços tratadas

In [50]:
df_endereco["rua"] = df_endereco["tipo_logr"] + " " + df_endereco["logr_nome"]
df_endereco["rua"] = df_endereco["rua"].str.lower().str.strip()
df_endereco.head()

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude,rua
1,Rua,João Antônio Vasconcelos,Rua João Antônio Vasconcelos - até 627/628,627,Amizade,Araçatuba,SP,16074320.0,-21.190276,-50.448686,rua joão antônio vasconcelos
413269,Rua,A,Rua A,NaN,Jardim Glória,Americana,SP,13468220.0,-22.756531,-47.343738,rua a
413270,Rua,A,Rua A,NaN,Jardim São Roque,Americana,SP,13469414.0,-22.743734,-47.348631,rua a
413271,Rua,A,Rua A,NaN,Iate Clube de Campinas,Americana,SP,13475620.0,-22.737846,-47.333569,rua a
413272,Rua,João Bandini,Rua João Bandini,NaN,Jardim Miriam,Americana,SP,13469072.0,-22.744283,-47.351588,rua joão bandini


### Dados geoespaciais

Para ler dados geoespaciais da base do censitária do IBGE precisamos importar a biblioteca geopandas. Além disso, os dados precisam ser baixados localmente e estarem localizados no mesmo diretório (https://drive.google.com/drive/folders/1CycoanzYN2oxKHPO6zxvIQ6Og1LBcujv) 

In [51]:
path_geo = ('/home/Rafaella.Duarte/Documentos/pasta_pessoal/precificacao_imoveis/Imersao_Dados/Large_Files/35SEE250GC_SIR.shp')
df_geo = gpd.read_file(path_geo)
df_geo.head()

,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
0,98237.0,354100005000009,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,1,"POLYGON ((-46.41024 -24.01103, -46.41169 -24.0..."
1,98232.0,354100005000004,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,2,"POLYGON ((-46.41606 -24.00245, -46.41598 -24.0..."
2,98230.0,354100005000002,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,3,"POLYGON ((-46.41213 -24.00792, -46.41244 -24.0..."
3,98229.0,354100005000001,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,4,"POLYGON ((-46.41134 -24.00552, -46.41196 -24.0..."
4,98231.0,354100005000003,URBANO,35410000500,None,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE SÃO PAULO,354100005001,Boqueirão,5,"POLYGON ((-46.41326 -24.00514, -46.41484 -24.0..."


Como estamos utilizando apenas dados de São Paulo, vamos seleciona-los

In [52]:
df_geo = df_geo[df_geo.NM_MUNICIP == "SÃO PAULO"]
df_geo.sample(3)

,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
46521,112061.0,355030820000047,URBANO,35503082000,None,355030820,CARRÃO,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,46522,"POLYGON ((-46.54577 -23.54455, -46.54609 -23.5..."
61806,127284.0,355030894000035,URBANO,35503089400,None,355030894,VILA SÔNIA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,61807,"POLYGON ((-46.75132 -23.59534, -46.75100 -23.5..."
48435,113928.0,355030829000015,URBANO,35503082900,None,355030829,FREGUESIA DO Ó,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,48436,"POLYGON ((-46.69436 -23.49005, -46.69433 -23.4..."


## Mesclando os dados

Com os dados devidamente tratados, vamos mescla-los para que a analise seja mais profunda. Os dataframes tratados são:

* df_imovel
* df_setor
* df_endereco
* df_geo

Vamos começar com os dados referentes aos endereços (df_endereço) e os dados dos valores dos imovéis (df_imove)

In [53]:
df_imovel.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira
2,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez
3,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto
5,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell


In [54]:
df_endereco.head()

,tipo_logr,logr_nome,logr_compl,numero,bairro,nome_localidade,sigla_uf,cep,latitude,longitude,rua
1,Rua,João Antônio Vasconcelos,Rua João Antônio Vasconcelos - até 627/628,627,Amizade,Araçatuba,SP,16074320.0,-21.190276,-50.448686,rua joão antônio vasconcelos
413269,Rua,A,Rua A,NaN,Jardim Glória,Americana,SP,13468220.0,-22.756531,-47.343738,rua a
413270,Rua,A,Rua A,NaN,Jardim São Roque,Americana,SP,13469414.0,-22.743734,-47.348631,rua a
413271,Rua,A,Rua A,NaN,Iate Clube de Campinas,Americana,SP,13475620.0,-22.737846,-47.333569,rua a
413272,Rua,João Bandini,Rua João Bandini,NaN,Jardim Miriam,Americana,SP,13469072.0,-22.744283,-47.351588,rua joão bandini


Realizando um left join, entre o 'df_imovel' e 'df_endereço', que retorna todos os registros do dataframe 'df_imovel' e os registros correspondentes do dataframe 'df_endereço' quando os valores em que os registros do campo 'apenas_rua' do dataframe 'df_imovel' for igual ao resgistro 'rua' do dataframe 'df_endereco'

In [55]:
df_imovel_endereco = pd.merge(
    left = df_imovel, 
    right = df_endereco[["rua", "cep", "latitude", "longitude"]], 
    how = "left", 
    left_on = "apenas_rua", 
    right_on = "rua"
).drop_duplicates(
    subset=df_imovel.columns
).query("cep > 0")

In [56]:
df_imovel_endereco

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,cep,latitude,longitude
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira,avenida itacira,11443160.0,-23.975993,-46.201317
5,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez,rua alba valdez,4845200.0,-23.752990,-46.695889
6,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN,NaN,18125000.0,-23.531043,-47.255023
25,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto,rua tobias barreto,13474663.0,-22.698901,-47.294461
38,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell,rua graham bell,4737030.0,-23.593796,-46.760077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89836,Avenida Odila,São Paulo,262,3,6,4,Planalto Paulista,1500000.0,avenida odila,avenida odila,4058020.0,-23.609093,-46.647258
89838,Rua Tapuçu,São Paulo,90,2,2,2,Vila Sofia,665000.0,rua tapuçu,rua tapuçu,4671185.0,-23.661018,-46.691521
89839,Rua Guararema,São Paulo,600,8,7,5,Bosque da Saúde,2300000.0,rua guararema,rua guararema,13030640.0,-22.922749,-47.076960
89856,Rua Estero Belaco,São Paulo,200,3,3,6,Vila da Saúde,1050000.0,rua estero belaco,rua estero belaco,4145020.0,-23.616124,-46.632983


Proximo passo será mesclar os dados do dataframe df_imovel_endereco (dataframe retornado da combinação entre df_imovel e df_endereço) com os dados do dataframe df_geo

In [57]:
df_geo.sample(3)

,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
54077,119571.0,355030853000065,URBANO,35503085300,None,355030853,MOOCA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,54078,"POLYGON ((-46.59016 -23.57666, -46.58981 -23.5..."
48476,113969.0,355030829000056,URBANO,35503082900,None,355030829,FREGUESIA DO Ó,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,48477,"POLYGON ((-46.70660 -23.49009, -46.70602 -23.4..."
58782,124307.0,355030876000345,URBANO,35503087600,None,355030876,SAPOPEMBA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,58783,"POLYGON ((-46.48568 -23.62059, -46.48519 -23.6..."


Criando um campo denominado 'Point' para armazenar a localização geografica (longitude e latitude) do imovél a venda.

In [58]:
df_imovel_endereco["Point"] = ""
for i in df_imovel_endereco.index:
    df_imovel_endereco["Point"][i] = Point(
        df_imovel_endereco["longitude"][i], 
        df_imovel_endereco["latitude"][i]
    )

/tmp/ipykernel_45668/1451948430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imovel_endereco["Point"][i] = Point(


In [59]:
df_imovel_endereco.sample(5)

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,cep,latitude,longitude,Point
48572,Rua Palmares,São Paulo,550,4,4,4,Brooklin Paulista,2900000.0,rua palmares,rua palmares,19800230.0,-22.663160,-50.404156,POINT (-50.4041555 -22.6631596)
9453,Rua Pirapó,São Paulo,572,4,5,8,Jardim Guedala,2500000.0,rua pirapó,rua pirapó,5610060.0,-23.584332,-46.709939,POINT (-46.7099389 -23.5843315)
31956,Rua Scuvero,São Paulo,156,3,2,3,Cambuci,700000.0,rua scuvero,rua scuvero,1527000.0,-23.563124,-46.626515,POINT (-46.6265155 -23.5631238)
71070,Rua Professor Francisco Maffei,São Paulo,453,3,7,6,Interlagos,2120000.0,rua professor francisco maffei,rua professor francisco maffei,4788080.0,-23.702388,-46.711311,POINT (-46.7113113 -23.7023879)
13591,NaN,São Paulo,146,3,4,2,Santo Amaro,1285000.0,NaN,NaN,18125000.0,-23.531043,-47.255023,POINT (-47.2550228 -23.5310431)


Criando um campo denominado setor_censo com o poligono na qual o ponto do imovel se localiza de acordo com os dados geoespaciais do IBGE

In [ ]:
df_imovel_endereco['setor_censo'] = df_imovel_endereco["Point"].map(
    lambda x: df_geo.loc[df_geo.contains(x), 'CD_GEOCODI'].values
).str[0]

In [61]:
df_imovel_endereco.sample(3)

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,cep,latitude,longitude,Point,setor_censo
84894,"Rua Pirandello, 785",São Paulo,450,3,5,3,Campo Belo,1750000.0,rua pirandello,rua pirandello,4623000.0,-23.634834,-46.674952,POINT (-46.6749517 -23.6348336),NaN
7058,NaN,São Paulo,534,6,7,7,Fazenda Morumbi,4300000.0,NaN,NaN,18125000.0,-23.531043,-47.255023,POINT (-47.2550228 -23.5310431),NaN
2873,Rua Professor Athanassof,São Paulo,150,3,2,2,Cidade Patriarca,430000.0,rua professor athanassof,rua professor athanassof,3552110.0,-23.535910,-46.505838,POINT (-46.5058377 -23.5359098),NaN


E por fim, apos mesclar os dataframes df_imovel, df_endereco e df_geo, vamos mesclar o dataframe retornado com o df_setor

In [62]:
df_setor.sample(3)

,Cod_setor,Cod_Grandes Regiï¿½es,Nome_Grande_Regiao,Cod_UF,Nome_da_UF,Cod_meso,Nome_da_meso,Cod_micro,Nome_da_micro,Cod_RM,...,V003,V004,V005,V006,V007,V008,V009,V010,V011,V012
9704,355030848000078,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,2.53,1.89,4684.02,30722137.39,5154.46,31385763.20,3230.45,21225493.43,4188.15,23512074.39
4344,355030825000309,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.43,2.12,690.48,210364.63,838.85,130447.59,444.68,238775.25,815.24,135176.15
1663,355030813000211,3,Regiï¿½o Sudeste,35,Sï¿½o Paulo,3515,Metropolitana de Sï¿½o Paulo,35061,Sï¿½o Paulo,20,...,3.76,2.22,883.53,355593.29,914.53,339435.65,535.61,359396.01,870.99,291691.50


In [63]:
df_imovel_endereco["setor_censo"] = df_imovel_endereco["setor_censo"].astype(float)

In [64]:
df_imovel_endereco["setor_censo"]

0                 NaN
5        3.550308e+14
6                 NaN
25                NaN
38       3.550309e+14
             ...     
89836             NaN
89838             NaN
89839             NaN
89856             NaN
89857             NaN
Name: setor_censo, Length: 6522, dtype: float64

In [65]:
df_imovel_endereco_setor = pd.merge(
    left = df_imovel_endereco, 
    right = df_setor, 
    how = "left", 
    left_on = "setor_censo", 
    right_on = "Cod_setor"
)

In [66]:
df_imovel_endereco_setor.head()

,Rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float,apenas_rua,rua,...,V003,V004,V005,V006,V007,V008,V009,V010,V011,V012
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0,avenida itacira,avenida itacira,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0,rua alba valdez,rua alba valdez,...,3.47,3.06,805.13,769188.78,1011.75,757433.42,518.77,426672.78,858.51,414378.88
2,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0,rua tobias barreto,rua tobias barreto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0,rua graham bell,rua graham bell,...,3.51,2.31,1026.36,272168.67,1143.96,167733.64,735.67,367088.08,1086.94,159622.46


In [67]:
df_imovel_endereco_setor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6522 entries, 0 to 6521
Data columns (total 48 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rua                    4367 non-null   object 
 1   cidade                 6522 non-null   object 
 2   metragem               6522 non-null   int64  
 3   quartos                6522 non-null   int64  
 4   banheiros              6522 non-null   int64  
 5   vagas                  6522 non-null   int64  
 6   bairro                 6522 non-null   object 
 7   preco_anuncio_float    6522 non-null   float64
 8   apenas_rua             4367 non-null   object 
 9   rua                    4367 non-null   object 
 10  cep                    6522 non-null   float64
 11  latitude               6522 non-null   float64
 12  longitude              6522 non-null   float64
 13  Point                  6522 non-null   object 
 14  setor_censo            6 non-null      float64
 15  Cod_